**量化后再训练**

常见有三种路径，难度和代价不同
- 冻结量化权重 + 只训适配器（LoRA / QLoRA / DoRA 等）：业界最常用；把基座权重量化到 8/4 bit 后不更新它们，只训练少量低秩适配器（通常 bf16/fp16）；稳定、省显存；
- QAT（Quantization-Aware Training，量化感知训练）端到端更新：在前向里插入伪量化（fake-quant）算子，用直通估计器（STE）反传，继续更新权重；一般先做 W8A8（权重与激活 8bit），W4A8/W4A4 难度显著上升；
- PTQ → 再小幅微调（PTQ 后轻微校正）：先做离线权重量化（如 GPTQ / AWQ），再用少量数据对输出头或少部分层做轻量校正 / 蒸馏，介于 1) 与 2) 之间；


**微调训练里都会量化哪些参数**

把大模型的计算图按部件拆开来看
- 权重（Weights）：Q / K / V / O 投影、MLP 的上 / 下投影（w1 / w2 / w3）是量化主力；
- 粒度常见：per-channel 或 group-wise（如每 64 / 128 列一组）；对称 / 非对称；可带零点；
- 激活（Activations）：QAT 或 W8A8 推理中会把中间激活做 8bit 伪量化；纯权重量化（AWQ / GPTQ）通常不量化激活；
- KV Cache（自回归生成时的缓存）：常做 8bit / 4bit（per-token / per-channel），显著省显存与带宽，对推理吞吐很关键；训练阶段一般仍用高精度，除非在做端到端的 QAT 并显式模拟；
- 嵌入层（token / pos）与 LM Head：容易对质量敏感，常保留 fp16 / bf16，或做 8bit（较少用 4bit）；
- 归一化与偏置（LayerNorm / RMSNorm、bias）：通常不量化，保留 fp16 / bf16；残差累加 / softmax 也保高精度以保证数值稳定；
- 梯度与优化器状态：即使用 QAT，梯度 / 动量多为 fp16 / bf16 /（少量场景 fp32）；为了省内存可用 8-bit 优化器状态（如 8-bit Adam），但这属于训练内存优化，不等同于模型权重/激活量化；

**常见训练路径对比怎么做**

一个是冻结量化权重 + 训练 LoRA（QLoRA 思路）
- 做法：把基座权重量化到 4bit（常见 NF4/FP4）或 8bit 并冻结；插入 LoRA 适配器（r = 16~64 常见），适配器 / 梯度 / 优化器保持 bf16 / fp16；
- 优点：显存开销小、稳定、实现简单，单卡消费级也能训；
- 代价：底座权重不更新，能力提升受 LoRA 容量限制；
- 适用：指令微调、领域适配、SFT / RLHF 的大多数落地场景；

端到端 QAT（W8A8 为主）
- 做法：插入 FakeQuant 模块（权重与激活），用 STE 反传；一般先在 fp16/bf16 预热几百/上千 step，再打开假量化；启动时用较小 LR 与分段/逐层启用以稳住训练。
- 推荐策略：
- 先从 W8A8 做起；
- LayerNorm / 嵌入 / 残差累加保 bf16；
- 结合校准 / 剪裁（percentile、Smooth 系数）抑制激活异常值； 
- 如追求更低比特（W4A8 / W4A4），常配合蒸馏或正则（如最小化量化前后输出的 MSE / KL）；
- 优点：可真正实现量化后继续学，并且最后得到对量化友好的权重；
- 代价：工程复杂，算力需求高，训练不稳风险更大；

PTQ + 轻量校正
- 做法：先做权重量化（GPTQ / AWQ 等），再用少量数据调整输出头或最后几层 / LoRA，或做最小化 logits 差异的蒸馏；
- 折中：实现简单、效果常优于纯 PTQ，代价远低于全面 QAT；

**设计细节要点和实操**
- 标定 / 校准数据：PTQ / QAT 都受分布影响；准备几百到几千条覆盖目标域的样本，用于动态范围统计、clipping；
- 量化策略：per-channel / 组量化通常比 per-tensor 稳定；组大小 64 / 128 是 LLM 的常见平衡点；
- 舍入与截断：最近邻、随机舍入、学习步长（LSQ）等；激活可配合 Smooth 类方法把离群值“转嫁”到权重尺度；
- 评测：不仅看 PPL / 准确率，还要看长上下文、推理链路、KV 命中、生成稳定性；不同任务对量化敏感点不同；

- 想省显存做效果可靠的微调：
	- 选 4bit 权重量化 + LoRA（QLoRA）；
	- LoRA r=16–64，α=16–64，target_modules 设到注意力与 MLP 线性层；
	- 不要量化 LayerNorm / RMSNorm、嵌入、残差；LM Head 若任务敏感，先保 bf16；
	- 可用 8-bit 优化器状态，再省一截显存；

- 要最终交付低比特可部署模型：
	- 先做 PTQ（AWQ/GPTQ） 得到 baseline；
	- 资源允许再做 W8A8 QAT（逐层 / 分段打开假量化），必要时对 logits 做蒸馏；
	- 生成端配 KV Cache 8bit / 4bit，观察长文本退化；

**对称量化（symmetric quantization）**

把数值按以 0 为中心、两边幅度相同的方式映射到整数网格；零点固定为 0，只需要一个尺度因子（scale）。

- 设比特数 $b$；带符号整数范围：$[-Q, Q]$，其中 $Q=2^{b-1}-1$（如 int8：$Q=127$）；
- 计算尺度：$s=\dfrac{\max|x|}{Q}$（可按 tensor／按通道／按组计算）；
- 量化：$q=\text{clip}\big(\text{round}(x/s),\,-Q,\,Q\big)$；
- 反量化：$\hat{x}=s\cdot q$；
- 零点（zero-point）= 0，因此不需要额外偏移项；

**和非对称量化对比**

| 项 | 对称 | 非对称 |
|---|---|---|
| 零点 | 固定 0 | 非零，可学习/校准 |
| 参数 | 只要 scale | scale + zero-point |
| 算子复杂度 | 简单（无偏移修正），核函数友好 | 需要补偿项 |
| 适配数据分布 | 适合围绕 0 对称或近似零均值的分布（如权重） | 适合偏正或非对称分布（如 ReLU 激活） |
| 精度风险 | 对强偏移分布更易失真 | 更鲁棒 |

**具体量化哪些部分时用它**
- 权重：常用对称（per-channel / group），稳定且高效；
- 激活：若经 ReLU 变成非负，常用非对称更好；若是带正负（如未 ReLU 或带规范化），对称可行；
- KV Cache：多为 8 / 4 bit，对称 / 非对称均有实现，取决于实现与分布；
- Norm / 偏置 / 残差累加：通常保持高精度（fp16 / bf16），不量化；

**校准与细节**
- 粒度：per-tensor（简单）、per-channel（更准）、group-wise（如每 64 / 128 列一组，折中）；
- 尺度估计：min-max 的 $\max|x|$，或百分位裁剪（如 99.9%）/ MSE 最优化，降低离群值影响；
- 范围选择：很多实现用 [-127,127] 的“严格对称”网格；也有用 int8 的 [-128,127] 并仍设零点为 0（略不对称，但工程常见）；
- QAT 中的反传：前向用上面的伪量化，反传用 STE（直通估计）近似 $\partial q/\partial x \approx 1$（在夹取区间内）；

In [5]:
import torch

x = torch.randn(1, 10)
b = 8

Q = 2**(b-1) - 1  # 127 for int8
s = x.abs().amax(dim=1, keepdim=True) / Q
q = (x / s).round().clamp(-Q, Q).to(torch.int8)
x_hat = q.to(torch.float32) * s

x, s, q, x_hat, x - x_hat

(tensor([[ 2.1110,  0.0346,  0.2623,  1.6478,  0.2410, -0.0840,  0.2816,  0.1709,
          -0.7608, -1.6636]]),
 tensor([[0.0166]]),
 tensor([[ 127,    2,   16,   99,   14,   -5,   17,   10,  -46, -100]],
        dtype=torch.int8),
 tensor([[ 2.1110,  0.0332,  0.2660,  1.6456,  0.2327, -0.0831,  0.2826,  0.1662,
          -0.7646, -1.6622]]),
 tensor([[ 0.0000,  0.0013, -0.0036,  0.0022,  0.0083, -0.0009, -0.0010,  0.0046,
           0.0038, -0.0014]]))